In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install opencv-python numpy filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 324.1 kB/s eta 0:00:0000:0100:01
donePreparing metadata (setup.py) ... 
done wheel for filterpy (setup.py) ... 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=4b1b77a3d65d7706fef8cadea8094f4c07cadf600d8f3656258ddb178913b8c9
  Stored in directory: /Users/alexander/Library/Caches/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

def predict_future_positions(kf, steps=35):
    future_positions = []
    state = kf.x.copy()  # Текущее состояние
    for _ in range(steps):
        state = np.dot(kf.F, state)  # Предсказание следующего состояния
        future_positions.append((int(state[0]), int(state[1]))) 
    return future_positions

model_path = "/Users/alexander/Downloads/best.pt"
model = YOLO(model_path)

# Настройка фильтра Калмана
kf = KalmanFilter(dim_x=4, dim_z=2)  
kf.F = np.array([[1, 0, 1, 0],  # Переходная матрица
                 [0, 1, 0, 1],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],  # Матрица измерений
                 [0, 1, 0, 0]])
kf.P *= 1000  
kf.R = np.array([[10, 0], [0, 10]])  
kf.Q *= 0.1 

video_path = "/Users/alexander/Downloads/IMG_9269.mp4"

# Открытие видео
cap = cv2.VideoCapture(video_path)

drone_tracks = []  
max_track_length = 35  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2

            # Обновление фильтра Калмана
            kf.predict()
            kf.update([x_center, y_center])
            x_pred, y_pred = kf.x[:2]  # Предсказанные координаты

            drone_tracks.append((int(x_pred), int(y_pred)))
            if len(drone_tracks) > max_track_length:
                drone_tracks.pop(0)

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    for i in range(1, len(drone_tracks)):
        cv2.line(frame, drone_tracks[i - 1], drone_tracks[i], (0, 0, 255), 2)

    # Предсказание будущих позиций
    future_positions = predict_future_positions(kf, steps=35)

    # Отрисовка предсказанной траектории
    for i in range(len(future_positions) - 1):
        cv2.line(frame, future_positions[i], future_positions[i + 1], (255, 0, 0), 2) 

    
    cv2.imshow("Drone Detection & Tracking", frame)



    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 640x352 1 drone, 55.3ms
Speed: 4.6ms preprocess, 55.3ms inference, 7.1ms postprocess per image at shape (1, 3, 640, 352)


/var/folders/d7/lmp8j75j3tl87d89s2cmcg900000gn/T/ipykernel_23170/300221554.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  drone_tracks.append((int(x_pred), int(y_pred)))
/var/folders/d7/lmp8j75j3tl87d89s2cmcg900000gn/T/ipykernel_23170/300221554.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  future_positions.append((int(state[0]), int(state[1])))



0: 640x352 1 drone, 62.6ms
Speed: 2.9ms preprocess, 62.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 75.5ms
Speed: 1.5ms preprocess, 75.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 236.6ms
Speed: 6.1ms preprocess, 236.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 60.1ms
Speed: 1.6ms preprocess, 60.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 73.7ms
Speed: 2.9ms preprocess, 73.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 352)



2025-03-25 18:45:41.369 python[23170:11223633] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-25 18:45:41.370 python[23170:11223633] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


0: 640x352 (no detections), 83.5ms
Speed: 5.2ms preprocess, 83.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 57.6ms
Speed: 1.2ms preprocess, 57.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 62.8ms
Speed: 2.8ms preprocess, 62.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 62.2ms
Speed: 1.2ms preprocess, 62.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 61.8ms
Speed: 1.1ms preprocess, 61.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 56.3ms
Speed: 2.9ms preprocess, 56.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 71.0ms
Speed: 1.2ms preprocess, 71.0ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 352)

0: 640x352 (no detections), 58.4ms
Speed: 2.4ms preprocess, 58.4ms in